In [2]:
import configparser
import json
import asyncio
from datetime import date, datetime

from telethon import TelegramClient
from telethon.errors import SessionPasswordNeededError
from telethon.tl.functions.messages import (GetHistoryRequest)
from telethon.tl.types import (
    PeerChannel
)
import numpy as np

#########################################################################################################################################
######################################################################################################################################
# some functions to parse json date
class DateTimeEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, datetime):
            return o.isoformat()

        if isinstance(o, bytes):
            return list(o)

        return json.JSONEncoder.default(self, o)


def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

#FUNCTION TO ENCODE BUY AND SELL
def orderTypeEncode(string):
    if string == 'BUY':
        return 0
    elif string == "SELL":
        return 1

#FUNCTION TO CONVERT PRICE TO POINTS
def priceToPoints(entry,another,symbol):
    if 'JPY' in symbol:
        point_value = 0.001
    else:
        point_value = 0.00001
    return np.int(np.abs(another-entry) / point_value)
 
def text_to_tradedict(text):

    symbol = text.split()[0]
    entry = float(deEmojify(text.split()[3]))
    SL = float(deEmojify(text.split()[5]))
    TP1 = float(deEmojify(text.split()[7]))
    TP2 = float(deEmojify(text.split()[9]))
    TP3 = float(deEmojify(text.split()[11]))

    my_trade1 = {}
    my_trade1['_action'] = 'OPEN'
    my_trade1['_type'] = orderTypeEncode(text.split()[1])
    my_trade1['_symbol'] = symbol
    my_trade1['_price'] = 0.0
    my_trade1['_SL'] = priceToPoints(entry,SL,symbol)
    my_trade1['_TP'] = priceToPoints(entry,TP1,symbol)
    my_trade1['_lots'] = 0.01
    my_trade1['_magic']= 123456
    my_trade1['_ticket']= 0

    my_trade2 = {}
    my_trade2['_action'] = 'OPEN'
    my_trade2['_type'] = orderTypeEncode(text.split()[1])
    my_trade2['_symbol'] = symbol
    my_trade2['_price'] = 0.0
    my_trade2['_SL'] = priceToPoints(entry,SL,symbol)
    my_trade2['_TP'] = priceToPoints(entry,TP2,symbol)
    my_trade2['_lots'] = 0.01
    my_trade2['_magic']= 123456
    my_trade2['_ticket']= 0

    my_trade3 = {}
    my_trade3['_action'] = 'OPEN'
    my_trade3['_type'] = orderTypeEncode(text.split()[1])
    my_trade3['_symbol'] = symbol
    my_trade3['_price'] = 0.0
    my_trade3['_SL'] = priceToPoints(entry,SL,symbol)
    my_trade3['_TP'] = priceToPoints(entry,TP3,symbol)
    my_trade3['_lots'] = 0.01
    my_trade3['_magic']= 123456
    my_trade3['_ticket']= 0

    return [my_trade1,my_trade2,my_trade3]

#############################################################################################################
#############################################################################################################
# Reading Configs
config = configparser.ConfigParser()
config.read("config.ini")

# Setting configuration values
api_id = config['Telegram']['api_id']
api_hash = config['Telegram']['api_hash']

api_hash = str(api_hash)

phone = config['Telegram']['phone']
username = config['Telegram']['username']

# Create the client and connect
client = TelegramClient(username, api_id, api_hash)



async def main(phone):
    await client.start()
    print("Client Created")
    # Ensure you're authorized
    if await client.is_user_authorized() == False:
        await client.send_code_request(phone)
        try:
            await client.sign_in(phone, input('Enter the code: '))
        except SessionPasswordNeededError:
            await client.sign_in(password=input('Password: '))

    me = await client.get_me()

    user_input_channel = input('enter entity(telegram URL or entity id):')

    if user_input_channel.isdigit():
        entity = PeerChannel(int(user_input_channel))
    else:
        entity = user_input_channel

    my_channel = await client.get_entity(entity)

    offset_id = 0
    limit = 1
    all_messages = []
    total_messages = 0
    total_count_limit = 1

    while True:
        print("Current Offset ID is:", offset_id, "; Total Messages:", total_messages)
        history = await client(GetHistoryRequest(
            peer=my_channel,
            offset_id=offset_id,
            offset_date=None,
            add_offset=0,
            limit=limit,
            max_id=0,
            min_id=0,
            hash=0
        ))
        if not history.messages:
            break
        messages = history.messages
        for message in messages:
            all_messages.append(message.to_dict())
        offset_id = messages[len(messages) - 1].id
        total_messages = len(all_messages)
        if total_count_limit != 0 and total_messages >= total_count_limit:
            break

    #with open('channel_messages.json', 'w') as outfile:
        #json.dump(all_messages, outfile, cls=DateTimeEncoder)
    #print(all_messages)
    latest_message_text = all_messages[0]['message']
    trade_dict_list = text_to_tradedict(latest_message_text)
    return trade_dict_list




####################################################################################
with client:
    everymess = client.loop.run_until_complete(main(phone))
    
    print('here is everymess variable')
    print(everymess)
    


RuntimeError: You must use "async with" if the event loop is running (i.e. you are inside an "async def")

In [2]:
import json
with open("channel_messages.json") as read_file:
    all_messages = json.load(read_file) #all_messages
    
    
    

In [4]:
all_messages[0]['id']

3

In [19]:
'BUY' in all_messages[0]['message']

True

In [13]:
all_messages[0]['_'] == "Message" #type = message
and 5 <= len(all_messages[0]['message'].split()[0]) <= 6 #first word length between 5 and 6
and ('BUY' in all_messages[0]['message'] or #message containing buy or sell
    'SELL' in all_messages[0]['message'])



{'_': 'Message',
 'id': 3,
 'to_id': {'_': 'PeerChannel', 'channel_id': 1475238870},
 'date': '2020-04-07T00:29:03+00:00',
 'message': 'GBPJPY BUY\n\nENTRY 134.18📍\n\nSL 133.72❌\n\nTP 134.38✅\nTP 134.68✅\nTP 135.18✅',
 'out': True,
 'mentioned': False,
 'media_unread': False,
 'silent': False,
 'post': True,
 'from_scheduled': False,
 'legacy': False,
 'edit_hide': False,
 'from_id': None,
 'fwd_from': None,
 'via_bot_id': None,
 'reply_to_msg_id': None,
 'media': None,
 'reply_markup': None,
 'entities': [],
 'views': 6,
 'edit_date': None,
 'post_author': None,
 'grouped_id': None,
 'restriction_reason': []}

In [20]:
all_messages[0]['_'] == "Message"

True